In [1]:
# #模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('Qwen/Qwen2.5-1.5B')

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-03 20:10:47,362 - modelscope - INFO - Target directory already exists, skipping creation.


In [1]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv("train_zh.csv", encoding='utf-8')
# df_split = np.array_split(df, 3)  # 拆分成 3 份

# # 保存拆分后的文件（可选）
# for i, split_df in enumerate(df_split):
#     split_df.to_csv(f"train_zh_part_{i+1}.csv", index=False, encoding='utf-8')
# # train_zh_part_1.csv (4045 rows)
# # train_zh_part_2.csv (4045 rows)
# # train_zh_part_3.csv (4045 rows)

/usr/local/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# import pandas as pd
# import numpy as np
# import csv
# # df=pd.read_csv("train_zh_part_1.csv", encoding='utf-8')
# # df
# max_length = 0
# with open('train_zh_part_1.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

# max_length = 0
# with open('train_zh_part_2.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

# max_length = 0
# with open('train_zh_part_3.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

最长的行数据字数为：5595
最长的行数据字数为：7482
最长的行数据字数为：5385


In [ ]:
# !python -m pip install --upgrade pip
# !pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install swanlab -i https://pypi.org/simple  # 使用官方PyPI源
!pip install swanlab -i https://repo.huaweicloud.com/repository/pypi/simple/
from swanlab.integration.transformers import SwanLabCallback

# config(need to run before training)

In [1]:
# -*- coding: utf-8 -*-
import torch
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset, load_dataset  # 正确的小写导入
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

# 基础配置
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "Qwen/Qwen2.5-1.5B" #参数量和 gpt2 一样.
basic_model_path = "./lora_basic"     # 基础微调结果
sft_model_path = "./lora_sft"        # SFT微调结果
# merged_model_path = "./lora_basic_sft"  # 合并模型
device_map = "auto"


# 加载模型和分词器
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# moda local model
model_name = "/mnt/workspace/.cache/modelscope/models/Qwen/Qwen2.5-1.5B"  # 替换为实际的本地模型路径

def tokenize_function(examples):
    # 假设 response 以 "<Response>" 开始
    response_start_token_id = tokenizer.encode("<Response>", add_special_tokens=False)[0]
    tokenized = tokenizer(
        examples["text"],
        padding="longest",  # 动态填充到批次中最长样本的长度
        # truncation=True,
        # max_length=512,
        truncation=False,   # 禁用截断
        return_tensors="pt"
    )
    labels = tokenized["input_ids"].clone()
    for i in range(len(labels)):
        # 找到 <Response> 的起始位置
        start_pos = (tokenized["input_ids"][i] == response_start_token_id).nonzero(as_tuple=True)[0]
        if len(start_pos) > 0:
            labels[i, :start_pos.item()] = -100  # 忽略 instruction + prompt 的 loss
    tokenized["labels"] = labels
    return tokenized

lora_config = LoraConfig(
    r=16,  # 增大 LoRA 矩阵秩 -------------------> next time to 8 #降低秩值
    lora_alpha=16,  # 调整 alpha 值 ------------------> next time to 32 #保持alpha/r=4的比例
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],,  # 扩展目标模块
    target_modules = ["c_attn", "mlp.down_proj", "mlp.up_proj"],  # qwen, 增强特征提取能力
    lora_dropout=0.1,  # 增加 dropout 防止过拟合 ---------------> next time to 0.2 #增加Dropout
    bias="none",
    task_type="CAUSAL_LM",
    # inference_mode=False,  # 确保处于训练模式[5](@ref)
    # modules_to_save=["lm_head"]  # 允许后续微调时更新头部[10](@ref)
)


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-04 15:05:51,029] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# basic knowledge(752 rows)

In [2]:
# ==================== 基础知识微调部分 ====================
# 加载基础知识数据集
def load_basic_knowledge_dataset(csv_path):
    df = pd.read_csv(csv_path)
    # 确保只有prompt和response列
    df = df[['prompt', 'response']]
    return Dataset.from_pandas(df)

# 基础知识数据预处理函数
def format_basic_data(example):
    prompt = example["prompt"]
    response = example["response"]
    full_prompt = f"<Prompt>{prompt}</Prompt>\n<Response>{response}</Response>"
    return {"text": full_prompt}

# 加载基础知识数据集
# basic_dataset = load_basic_knowledge_dataset("game_strategy_and_term.csv")
basic_dataset = load_dataset("csv", data_files="game_strategy_and_term.csv")["train"]
# print(basic_dataset["train"])
# basic_dataset = basic_dataset.select(range(50))  # 选择前5个样本
basic_dataset = basic_dataset.map(format_basic_data, remove_columns=["prompt", "response"])
basic_dataset = basic_dataset.train_test_split(test_size=0.1)

# # # 加载前10行数据
# # def load_mini_dataset(csv_path, n_rows=2):
# #     df = pd.read_csv(csv_path, nrows=n_rows)
# #     # print(df)
# #     return Dataset.from_pandas(df)

# # 加载迷你数据集
# # dataset = load_mini_dataset("train_zh.csv")
# # dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])
# # # 加载数据集
# dataset = load_dataset("csv", data_files="train_zh.csv")["train"]
# dataset = dataset.select(range(5))  # 选择前5个样本 [[7]] for test.
# dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])


In [3]:
basic_dataset['train']['text'][0]

'<Prompt>狼美人在狼人杀游戏中应该如何发挥其特殊技能“魅惑殉情”以获得胜利？</Prompt>\n<Response>在狼人杀游戏中，狼美人是一个具有特殊技能的角色。她的技能是“魅惑殉情”，可以与另一名玩家绑定，若狼美人在游戏中出局，该玩家也会随之出局。为了帮助狼人杀新手玩家更好地了解狼美人的玩法，本文将详细介绍狼美人的玩法攻略。\n1、了解狼美人的技能和胜利条件\n狼美人的技能是“魅惑殉情”，可以将该技能使用在神职玩家身上，以发挥其最大效果。否则，如果只是杀死一个平民，对于狼美人来说就没有存在的必要了。狼美人的胜利条件与狼人一致，需要将所有神职玩家淘汰出局。\n2、绑定强神\n狼美人想要发挥出最大的效果，就需要绑定强神。在游戏初期，强神中的预言家会起跳，因此狼队可以选择抗推或者夜晚刀掉预言家。而女巫、守卫和猎人是强神中的重要角色，狼美人应该优先将他们作为魅惑对象。为了成功绑定强神，狼美人需要具备一定的抿神能力。\n3、优先选择魅惑对象\n狼美人需要学会隐藏自己的身份，并将自己的魅力对象选择为强大的神职角色，如女巫和守卫。在游戏初期，狼美人不需要悍跳神牌，只需暗中观察找到预言家、女巫和守卫中的一个，魅力优先级：女巫>守卫>预言家。下一步就是想方设法让自己出局把迷人的对象带走，这样可以给狼队增加一个轮次。\n4、与狼队友共享信息\n在游戏过程中，狼美人需要与狼队友共享迷惑的信息。如果狼美人没有连中神牌，狼队友可以选择自爆，创造新的机会让狼美人再次迷惑，间接给狼队增加新的轮次。\n5、学会伪装\n在发言时，狼美人需要将自己描述为普通狼人的样子。同时，需要发挥自己是狼人的视角优势，把自己打成焦点牌，但能故意聊得很差，让好人把你标为普通狼人票出局，这样你就能顺利带走强神！\n总之，在狼人杀游戏中扮演狼美人时，你需要掌握以上攻略，以便更好地发挥自己的特殊技能。通过观察和抿神的能力，选择正确的魅惑对象和时机，与狼队友保持沟通并学会伪装自己。只有这样，才能在游戏中获得胜利！</Response>'

In [4]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

# 准备模型
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# # 基础知识微调训练
# print("开始基础知识微调训练...")
# basic_tokenized = basic_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# tokenized_dataset = dataset.map(
#     tokenize_function,
#     batched=True,
#     remove_columns=["text"]
# )


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


trainable params: 9,404,416 || all params: 1,553,118,720 || trainable%: 0.6055


In [32]:
# print(tokenized_dataset["train"])  # First 5 training samples

In [5]:
# 基础知识微调训练
print("开始基础知识微调训练...")
basic_tokenized = basic_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

basic_training_args = TrainingArguments(
    output_dir=basic_model_path,
    num_train_epochs=3,  # 基础知识微调可以使用较少的epoch
    per_device_train_batch_size=2,     # -> next time to 4
    gradient_accumulation_steps=4,      # -> next time to 8   # 等效批量=32
    learning_rate=2e-4,  # 基础知识微调可以使用稍高的学习率
    fp16=True,
    eval_steps=10,
    logging_steps=1,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=50,
)
basic_trainer = Trainer(
    model=model,
    args=basic_training_args,
    train_dataset=basic_tokenized["train"],
    eval_dataset=basic_tokenized["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)
basic_trainer.train()
print("基础知识微调完成!")

# 新增保存逻辑
# basic_model_path = "./lora_basic"  # 指定基础微调保存路径
model.save_pretrained(basic_model_path)  # 保存LoRA适配器参数[8](@ref)
tokenizer.save_pretrained(basic_model_path)

开始基础知识微调训练...


Map: 100%|██████████| 76/76 [00:00<00:00, 2062.30 examples/s]
Detected kernel version 4.19.91, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details 

Step,Training Loss,Validation Loss
10,2.983100,3.026573
20,2.997700,2.866308
30,2.927200,2.788158
40,2.430700,2.732572
50,2.526800,2.689067
60,3.046700,2.664482
70,2.816600,2.631630
80,2.570300,2.604075
90,2.321900,2.590086
100,2.316100,2.572315


/usr/local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/site-packages/torch/_dynamo/

基础知识微调完成!


('./lora_basic/tokenizer_config.json',
 './lora_basic/special_tokens_map.json',
 './lora_basic/vocab.json',
 './lora_basic/merges.txt',
 './lora_basic/added_tokens.json',
 './lora_basic/tokenizer.json')

# SFT (12134 rows, train 10921, test 10%)

In [2]:
# 在SFT训练代码前重新初始化模型并加载基础微调结果：
# 重新初始化基础模型（重要！）
# 加载模型和分词器
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_compute_dtype=torch.float16  # 强制使用 FP16 计算
)
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 从基础微调保存路径加载
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # gradient_checkpointing=True,#启用梯度检查点
    device_map=device_map,
    trust_remote_code=True
)
base_model = prepare_model_for_kbit_training(base_model)
# 加载第一阶段LoRA参数
model = get_peft_model(base_model, LoraConfig.from_pretrained(basic_model_path))
model.print_trainable_parameters()  # 验证参数加载

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


trainable params: 0 || all params: 1,553,118,720 || trainable%: 0.0000


In [3]:
# ==================== SFT微调部分 ====================
# SFT数据预处理函数
def format_sft_data(example):
    system_prompt = example["instruction"]
    user_input = example["prompt"]
    response = example["response"]
    full_prompt = f"<Instruction>{system_prompt}</Instruction>\n<Prompt>{user_input}</Prompt>\n<Response>{response}</Response>"
    return {"text": full_prompt}

# 加载SFT数据集
sft_dataset = load_dataset("csv", data_files="train_zh.csv")["train"]
# print(sft_dataset)
sft_dataset = sft_dataset.select(range(5))  # 选择前5个样本
sft_dataset = sft_dataset.map(format_sft_data, remove_columns=["instruction", "prompt", "response", "meta"])
sft_dataset = sft_dataset.train_test_split(test_size=0.1)

In [4]:
sft_dataset['train']['text'][0]
# print(tokenizer.model_max_length)  # 查看模型支持的最大长度（如512）

'<Instruction>\n你现在正在玩一种叫做“狼人杀”的游戏。\n在这款游戏中，玩家通常被分为两个阵营：狼人和村民。\n狼人杀游戏中不同角色的玩家有不同的目标：\n- 村民的目的是识别出狼人，并通过投票使他们出局。\n- 对于狼人来说，他们的主要目标是隐藏他们的真实身份，在讨论中误导他人，以免被投票出局并尽可能的猎杀村民。\n以下是一些基本规则：\n- 身份：玩家的身份是秘密分配的。狼人彼此知道对方的身份，而村民只知道自己的身份。\n- 昼夜更替：游戏有交替的白天和黑夜阶段。夜里，狼人秘密选择一名村民猎杀。白天，所有玩家讨论并投票决定他们认为是狼人的玩家，票数最多的玩家被淘汰。\n- 特殊角色：游戏中有存在一些有特殊能力的角色，比如能得知玩家身份的“预言家”等。\n- 获胜条件：当游戏中有一个群体实现它们的获胜条件时游戏结束。如果所有狼人被淘汰，村民就获胜。如果狼人杀死了所有普通村民或所有特殊角色，狼人就获胜。\n\n在这个游戏中，我们有从1到7号共7名玩家 —— 5名村民和2名狼人。村民中有特殊角色，包括：\n- 1位预言家：\n    - 目标：预言家的目的是帮助村民识别狼人。\n    - 能力：在夜晚阶段，预言家可以秘密选择一名玩家，每晚了解他的真实身份（是否为狼人）。\n- 1位守卫：\n    - 目标：守卫的目的是策略性地使用他的特殊能力来帮助村民。\n    - 能力：守卫每晚可以保护一名玩家，防止他们受到狼人的攻击。守卫可以选择保护自己，或者选择不保护任何人，但他不能在连续两个夜晚保护同一个玩家。\n其他的都是普通村民。\n</Instruction>\n<Prompt>在本场游戏中，你目前已知以下信息：\n1. 角色设定：\n你是3号玩家。\n你的身份是：预言家。\n你每晚可以查看一名玩家是否为狼人，你的目标是利用这些信息帮助其他人找出并淘汰所有狼人。\n2. 客观信息：\n- 游戏进程：目前游戏进行到第2轮。\n- 当前存活的玩家有：1号，3号，4号，5号，6号，只能在以上玩家中选择进行查验\n- 行动记录：第1轮预言家查验2号玩家，2号玩家是狼人。\n- 投票情况：第1轮投票记录：1号玩家投给：2号玩家；\n2号玩家投给：3号玩家；\n3号玩家投给：2号玩家；\n4号玩家投给：2号玩家；\n5号玩家投给：2号玩家；\n6号玩家投给：2

# 爆显存, 需要将数据集 cut 一半试试, 主要 response 不能够被截断, 所以不能 truncation. 最长的输入序列是 7482.

In [5]:
# # 在基础知识训练结束后保存适配器参数
# basic_lora_weights = model.lora_A.weight.detach().clone()

# # 在SFT训练开始前加载对比
# assert torch.allclose(model.lora_A.weight, basic_lora_weights), "参数未继承！"
from torch.cuda.amp import GradScaler
scaler = GradScaler()  # 创建梯度缩放器

# SFT微调训练
def tokenize_function(examples):  # 新增参数
    # 假设 response 以 "<Response>" 开始
    response_start_token_id = tokenizer.encode("<Response>", add_special_tokens=False)[0]
    # print("Response token ID:", response_start_token_id)  # 应为有效数值
    tokenized = tokenizer(
        examples["text"],
        # padding="longest",  # 动态填充到批次中最长样本的长度
        padding="max_length",
        truncation='only_first',      # 禁用截断（需确保所有样本长度 ≤ max_length）
        max_length=7500,
        return_tensors="pt"
    )
    labels = tokenized["input_ids"].clone()
    for i in range(len(labels)):
        # 找到 <Response> 的起始位置
        start_pos = (tokenized["input_ids"][i] == response_start_token_id).nonzero(as_tuple=True)[0]
        if len(start_pos) > 0:
            labels[i, :start_pos.item()] = -100  # 忽略 instruction + prompt 的 loss
    tokenized["labels"] = labels
    return tokenized

print("开始SFT微调训练...")
sft_tokenized = sft_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

sft_training_args = TrainingArguments(
    output_dir=sft_model_path,  # 新保存路径
    num_train_epochs=5,  # 增加训练轮次
    per_device_train_batch_size=2,  # 增大批量大小
    gradient_accumulation_steps=2,  # 增加梯度累积步数
    learning_rate=2e-4,  # 调整学习率
    fp16=True,
    max_grad_norm=1.0,
    fp16_full_eval=False,  # 禁用评估阶段的混合精度
    gradient_checkpointing=True,  # 启用梯度检查点优化显存[4](@ref)
    eval_steps=10,  # 降低评估步数以适配小数据量
    logging_steps=1,      # 每个训练步骤记录日志
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    # logging_dir="./logs",  # 新增TensorBoard日志目录
    # report_to=["tensorboard"],  # 启用TensorBoard报告
    # load_best_model_at_end=True  # 自动加载最佳模型
)

# swanlab_callback = SwanLabCallback()

# 创建Trainer
sft_trainer = Trainer(
    model=model,
    args=sft_training_args,
    train_dataset=sft_tokenized["train"],
    eval_dataset=sft_tokenized["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    # callbacks=[swanlab_callback]
)
torch.autograd.set_detect_anomaly(True)  # 开启梯度异常检测
sft_trainer.train()

# 保存最终模型
# model.save_pretrained("./lora_final")
tokenizer.save_pretrained(sft_model_path)
print(f"模型已保存至 {sft_model_path}")



/tmp/ipykernel_13668/2130614555.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # 创建梯度缩放器


开始SFT微调训练...


Map: 100%|██████████| 1/1 [00:00<00:00, 139.37 examples/s]
Detected kernel version 4.19.91, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


swanlab: Tracking run with swanlab version 0.5.7                                   
swanlab: Run data will be saved locally in /mnt/workspace/llm-project/werewolf_game_reasoning/swanlog/run-20250504_150608-6c031199
swanlab: 👋 Hi ckkai, welcome to swanlab!
swanlab: Syncing run ./lora_sft to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@ckkai/werewolf_game_reasoning
swanlab: 🚀 View run at https://swanlab.cn/@ckkai/werewolf_game_reasoning/runs/tjotpx87zgzwpg9bwr2k2



[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.49 GiB. GPU 0 has a total capacity of 22.20 GiB of which 2.95 GiB is free. Process 106081 has 19.25 GiB memory in use. Of the allocated memory 17.73 GiB is allocated by PyTorch, and 1.25 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [26]:
# for log in trainer.state.log_history:
#     print(log)
# # 提取训练日志（包含 loss 或 train_loss 和 step）
# train_logs = []
# for log in trainer.state.log_history:
#     if "step" in log:
#         if "loss" in log:
#             log["train_loss"] = log["loss"]  # 统一字段名为 train_loss
#         train_logs.append(log)

# # 转换为 DataFrame
# train_df = pd.DataFrame(train_logs)[["step", "train_loss"]]
# # 提取评估日志（包含 eval_loss 和 step）
# eval_logs = [log for log in trainer.state.log_history if "eval_loss" in log and "step" in log]
# eval_df = pd.DataFrame(eval_logs)[["step", "eval_loss"]] if eval_logs else pd.DataFrame()
# # 合并训练和评估日志
# merged = pd.merge(train_df, eval_df, on="step", how="outer").sort_values("step")
# merged.ffill(inplace=True)  # 使用 ffill() 替代 fillna(method="ffill")
# plt.figure(figsize=(12, 6))
# plt.plot(merged["step"], merged["train_loss"], 'b-', label='Training Loss')
# if not eval_df.empty:
#     plt.plot(merged["step"], merged["eval_loss"], 'r--', label='Validation Loss')
# plt.title("Training Progress Analysis")
# plt.xlabel("Training Steps")
# plt.ylabel("Loss Value")
# plt.legend()
# plt.grid(True)
# plt.show()

# 生成任务评估（BLEU/ROUGE/METEOR）

In [12]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import jieba  # 中文分词支持

# 加载原始模型和微调模型
base_model = AutoModelForCausalLM.from_pretrained(model_name)
# finetuned_model = PeftModel.from_pretrained(base_model, output_dir)
finetuned_model = PeftModel.from_pretrained(base_model, basic_model_path)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载测试数据
df = pd.read_csv("train_zh.csv").tail(2)
# df = pd.read_csv("game_strategy_and_term.csv").tail(2)
test_dataset = Dataset.from_pandas(df)


In [13]:
import logging
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import jieba
from torch.nn import CrossEntropyLoss

# 关闭jieba调试日志
logging.getLogger("jieba").setLevel(logging.WARNING)

# 中文分词工具（改用搜索引擎模式提升召回率）
def chinese_tokenize(text):
    return list(jieba.cut_for_search(text))  # 使用搜索引擎模式[[2]]

# 生成预测文本（增加生成长度）
def generate_response(model, instruction, prompt):
    input_text = f"[INST] <<SYS>>\n{instruction}\n<</SYS>>\n\n{prompt} [/INST]"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)  # 增加生成长度[[1]]
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # 生成预测文本（增加生成长度）
# def generate_response(model, instruction, prompt):
#     input_text = f"[INST] <<SYS>>\n{instruction}\n<</SYS>>\n\n{prompt} [/INST]"
#     inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens=100)  # 增加生成长度[[1]]
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 显式设置pad_token_id（避免警告）
tokenizer.pad_token_id = tokenizer.eos_token_id

# 计算BLEU/ROUGE（增加平滑函数和ROUGE-L）
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
smoother = SmoothingFunction()

def calculate_perplexity(model, tokenizer, prompt, response):
    """
    计算模型对给定prompt和response的困惑度
    """
    full_text = prompt + " " + response  # 拼接输入与响应
    inputs = tokenizer(full_text, return_tensors="pt").to(model.device)
    
    # 构造标签：仅计算response部分的loss
    prompt_len = len(tokenizer(prompt, add_special_tokens=False)['input_ids'])
    labels = inputs['input_ids'].clone()
    labels[:, :prompt_len] = -100  # 忽略prompt部分的loss计算
    
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
    
    return torch.exp(loss).item()  # 返回困惑度

for i, example in enumerate(test_dataset):
    instruction = example["instruction"]
    # print(f"指令: {instruction}")
    prompt = example["prompt"]
    # print(f"输入: {prompt}")
    reference = example["response"]
    # print(f"参考答案: {reference}")
    
    # 生成预测
    base_output = generate_response(base_model, instruction, prompt)
    ft_output = generate_response(finetuned_model, instruction, prompt)
    # 新增：计算困惑度
    base_ppl = calculate_perplexity(base_model, tokenizer, prompt, reference)
    ft_ppl = calculate_perplexity(finetuned_model, tokenizer, prompt, reference)
    
    # 分词处理
    ref_tokens = chinese_tokenize(reference)
    base_tokens = chinese_tokenize(base_output)
    ft_tokens = chinese_tokenize(ft_output)
    
    # BLEU优化：使用BLEU-2+平滑函数
    bleu_base = sentence_bleu(
        [ref_tokens], base_tokens, 
        weights=(0.5, 0.5),  # 使用BLEU-2
        smoothing_function=smoother.method1  # 添加平滑
    )
    bleu_ft = sentence_bleu(
        [ref_tokens], ft_tokens,
        weights=(0.5, 0.5),
        smoothing_function=smoother.method1
    )
    
    # ROUGE优化：使用分词后的文本并增加ROUGE-L
    rouge_base = scorer.score(
        " ".join(ref_tokens), 
        " ".join(base_tokens)
    )
    rouge_ft = scorer.score(
        " ".join(ref_tokens), 
        " ".join(ft_tokens)
    )
    # 新增打印内容：对比生成结果与参考答案
    print(f"\n{'='*20} 样本{i+1} 原始内容 {'='*20}")
    # print(f"【指令】: {instruction}")
    # print(f"【输入】: {prompt}")
    print(f"【参考回答】: {reference}")
    print(f"\n【基础模型生成】: {base_output}")
    print(f"【微调模型生成】: {ft_output}")
    
    # 评估指标打印（保持原有格式）
    print(f"\n{'='*20} 样本{i+1} 评估结果 {'='*20}")
    print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    print("-"*50 + "\n")
    
    # 打印困惑度结果
    print(f"Perplexity - 原始: {base_ppl:.2f}, 微调: {ft_ppl:.2f}")
    print("-"*50 + "\n")
    
    # print(f"样本{i+1}评估结果:")
    # print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    # print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    # print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    # print("-"*50)

KeyError: 'instruction'